In [ ]:
import os
import sys
import glob, pickle
import numpy as np
import pandas as pd

#Import audio packages
import librosa
import librosa.display
from scipy.io import wavfile
import scipy.io.wavfile
import sys

#Import plotting packages
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
import matplotlib.pyplot as plt
import seaborn as sns
#Import Keras & Tensorflow packages
import keras
from keras import regularizers
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split


emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['neutral','calm', 'happy', 'sad', 'angry', 'fearful', 'disgust','surprised']


def extract_feature(file_name, mfcc):
        X,sample_rate = librosa.load(os.path.join(file_name), res_type='kaiser_fast')
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack(mfccs)
        return result
def dataset_options():
    ravdess = True
    data = {'ravdess':ravdess}
    return data

def load_data(test_size=0.2):
    x,y = [],[]
    mfcc=True
    
    data = dataset_options()
    paths=[]
    path = paths.append() #(load the path of dataset....\Actor_*\\*.wav")
    
    for path in paths:
        for file in glob.glob(path):
            file_name=os.path.basename(file)
            emotion=emotions[file_name.split("-")[2]]
            if emotion not in observed_emotions:
                continue
            feature=extract_feature(file, mfcc)
            x.append(feature)
            y.append(emotion)
    return {"X":x,"y":y}


#PLOTTING AN AUDIO WAVE FROM DATASET

#librosa.core.load(path, sr=22050, mono=True, offset=0.0, duration=None, dtype=<class 'numpy.float32'>, res_type='kaiser_best')
res_type_s = 'kaiser_best'
duration_s = None
sample_rate_s = 22050
offset_s = 0.5

# y = audio time series
# sample_rate = sampling rate of y

y, sample_rate = librosa.load() #load the path of a wave from dataset..........\Actor_03/03-01-01-01-02-02-03.wav, 
                                  res_type = res_type_s,
                                  duration = duration_s,
                                  sr = sample_rate_s,
                                  offset = offset_s,
                                 mono=False)
                                 
import os
import pandas as pd
import librosa
import glob 

plt.figure(figsize=(15, 5))
librosa.display.waveplot(y, sr=sample_rate)


#PLOTTING MFCC OF THE ABOVE AUDIO WAVE EXAMPLE

librosa_audio, librosa_sample_rate = librosa.load() #load the path of a wave from dataset....\Actor_03/03-01-01-01-02-02-03.wav

mfccs = librosa.feature.mfcc(y=librosa_audio, sr=librosa_sample_rate, n_mfcc=40)
print(mfccs.shape)

import librosa.display
librosa.display.specshow(mfccs, sr=librosa_sample_rate, x_axis='time')

#FEATURES

import time
start_time=time.time()
Trail_dict = load_data(test_size=0.2)
X = pd.DataFrame(Trail_dict["X"])
y = pd.DataFrame(Trail_dict["y"])
data = pd.concat([X,y],join='outer',axis=1)
data.columns=['feature1','feature2','feature3','feature4','feature5','feature6','feature7','feature8','feature9','feature10','feature11',
             'feature12','feature13','feature14','feature15','feature16','feature17','feature18','feature19','feature20','feature21','feature22',
              'feature23',
             'feature24','feature25','feature26','feature27','feature28','feature29','feature30','feature31','feature32',
              'feature33','feature34','feature35','feature36','feature37','feature38','feature39','feature40','emotion'
             ]
data['emotion']=data['emotion'].astype('object')
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
data['emotion']=le.fit_transform(data['emotion'])
data.head()


#SPLITTING DATASET (TRAINING AND TESTING)

from sklearn.model_selection import train_test_split

#Split train & test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=42)

# Check out the data
print(f'X_train shape: {X_train.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_test shape: {y_test.shape}')


#LABLE ENCODING

from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder

lb = LabelEncoder()

#Encode emotion labels into numbers
y_train_lb = np_utils.to_categorical(lb.fit_transform(y_train))
y_test_lb = np_utils.to_categorical(lb.fit_transform(y_test))

# Check out the data
print(f'X_train shape: {X_train.shape}')
print(f'y_train shape: {y_train_lb.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_test shape: {y_test_lb.shape}')


#Check encoding
np.unique(y_train_lb, axis=0)


#Check encoding labels
print(lb.classes_)


#Building new lists of encoding labels
y_labels_encoded = {}
for i, label in enumerate(lb.classes_):
    y_labels_encoded[i] = label
    
print(y_labels_encoded)


from sklearn.preprocessing import StandardScaler
#SCALING THE DATA
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scalled = scaler.transform(X_train)
X_test_scalled = scaler.transform(X_test)


#Adding dimension for CNN
x_traincnn = np.expand_dims(X_train_scalled, axis=2)
x_testcnn = np.expand_dims(X_test_scalled, axis=2)

#Checking shapes of dataframes
print(x_traincnn.shape)
print(x_testcnn.shape)

#Importing packages for CNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Conv1D 
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, BatchNormalization, Flatten, MaxPooling2D
from keras.models import Model
from keras.callbacks import ModelCheckpoint




#BUILDING CNN MODEL
CNN_model = Sequential()

#Build first layer
CNN_model.add(Conv1D(16, 5,padding='same',
                 input_shape=(40, 1), activation='relu'))
CNN_model.add(Dropout(0.2))
CNN_model.add(MaxPooling1D(pool_size=(2)))

#Build second layer
CNN_model.add(Conv1D(32, 5,padding='same',activation='relu'))
CNN_model.add(Dropout(0.2))
CNN_model.add(MaxPooling1D(pool_size=(2)))
#Build third layer
CNN_model.add(Conv1D(64, 5,padding='same',activation='relu'))
CNN_model.add(Dropout(0.3))
CNN_model.add(MaxPooling1D(pool_size=(2)))
#Build forth layer
CNN_model.add(Conv1D(128, 5,padding='same',activation='relu'))
CNN_model.add(Dropout(0.5))
CNN_model.add(MaxPooling1D(pool_size=(2)))
#Add dropout
CNN_model.add(Dropout(0.3))

#Flatten 
CNN_model.add(Flatten())

CNN_model.add(Dense(128, activation ='relu'))
CNN_model.add(Dropout(0.1))
CNN_model.add(Dense(64, activation ='relu'))
CNN_model.add(Dense(8, activation='softmax'))
# CNN model summary
CNN_model.summary()



# COMPILING THE MODEL 
CNN_model.compile(loss = 'categorical_crossentropy',
                  optimizer = 'Adam',
                  metrics = ['accuracy'])
from keras.callbacks import ModelCheckpoint 

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_cnn.hdf5', 
                               verbose=1, save_best_only=True)



#MODEL FIT
cnn_results = CNN_model.fit(x_traincnn, y_train_lb,
              batch_size = 64,
              epochs = 25,
              verbose = 1,
              validation_data = (x_testcnn, y_test_lb))



#Ploting model accuracy over ephocs
plt.plot(cnn_results.history['accuracy'])
plt.plot(cnn_results.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()



# Evaluating the model on the training and testing set
score = CNN_model.evaluate(x_traincnn, y_train_lb, verbose=0)
print("Training Accuracy: ", score[1])

score = CNN_model.evaluate(x_testcnn, y_test_lb, verbose=0)
print("Testing Accuracy: ", score[1])


#Get predictions from model
y_test_predictions = CNN_model.predict_classes(x_testcnn)
print(y_test_predictions)


#Get labels for emotions
print(y_labels_encoded)




from sklearn.metrics import confusion_matrix

#BUILDING CONFUSION MATRIX 
confusion_matrix = confusion_matrix(y_test, y_test_predictions_labels)
print(confusion_matrix)


#PLOTTING HEAT MAP
ax = sns.heatmap(confusion_matrix, annot=True)



#Adding labels to confusion matrix
confusion_matrix = pd.DataFrame(confusion_matrix, columns=list(y_labels_encoded.values()), index=list(y_labels_encoded.values()))

print("The rows represents the true values or observations")
print("The columns respresent the model's predictions")
print(confusion_matrix)